### 0. Loading dependencies

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as scipy
import statsmodels.formula.api as smf
import random
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import iqr
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import warnings
# String styling
class color:
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

/home/tshepo/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version



### 1.1 Data Preparation: Data Cleaning, Relabeling, Wrangling and splitting subset.

In [2]:
#Fill Nan values is salary column
df_salary = pd.read_csv('dataset/salary.csv')
df_salary = df_salary.fillna(df_salary.salary.mean())

#Rename as per metadata
df_salary =df_salary.rename(columns ={'male':'gender', 'experior':'Experience', 'yearsabs':'years_absent'})
df_salary.gender = ['male' if i == 1 else 'female' for i in df_salary.gender]

df_salary.position = ['Jnr_employee' if i == 1 else 'Manager' if  i == 2 
                      else 'Executive' for i in df_salary.position]

df_salary.Field = ['Engineering' if i == 1 else 'Finance' if i == 2
                   else 'Human Resource' if i == 3 else 'Marketing' 
                   for i in df_salary.Field]    
df_salary = pd.DataFrame(df_salary)
df_salary.head()

FileNotFoundError: [Errno 2] File dataset/salary.csv does not exist: 'dataset/salary.csv'

In [ ]:
#Splitting the data into df_train train and test set

df_train, df_test = train_test_split(df_salary, test_size=0.2, random_state=42)
df_train

## Analyze relationships

In [ ]:
sns.pairplot(df_train)
plt.show()

## Checking and handling outliers

In [ ]:
# Quartiles and IQR:df_train
q75_train, q25_train = np.percentile(df_train['salary'], [75 ,25])
iqr_train = q75_train - q25_train

# Extremes' boundaries: df_train
lower_bound = q25_train - iqr_train*1.5
upper_bound = q75_train + iqr_train*1.5

#Determining if an entry is an extreme or an outlier:df_train
df_train['Is_it_an_extreme'] = ['Yes' if i < lower_bound else 'Yes' 
                                if i > upper_bound else "No" 
                                for i in df_train.salary]
df_outliers = df_train[df_train.Is_it_an_extreme == 'Yes']

#Dropping outliers
df_train = df_train[df_train.Is_it_an_extreme != 'Yes']

#df_train descriptive statistics
# df_train.describe()

### 2. Hot encoding Dummy variables for Field variable with Human Resource, Position with Executive and Gender with Females as references respectively

In [ ]:
df_dummies =pd.get_dummies(df_train.Field).loc[:,['Engineering','Finance','Marketing']]
df_dummies1 =pd.get_dummies(df_train.position).loc[:,['Jnr_employee','Manager']]
df_dummies2 =pd.get_dummies(df_train.gender).loc[:,['male']]

df_train = df_train.join(df_dummies)
df_train = df_train.join(df_dummies1)
df_train = df_train.join(df_dummies2)

df_dummies3 =pd.get_dummies(df_test.Field).loc[:,['Engineering','Finance','Marketing']]
df_dummies4 =pd.get_dummies(df_test.position).loc[:,['Jnr_employee','Manager']]
df_dummies5 =pd.get_dummies(df_test.gender).loc[:,['male']]
df_test = df_test.join(df_dummies3)
df_test = df_test.join(df_dummies4)
df_test = df_test.join(df_dummies5)

### 3. correlation matrix comparing the relationship of salary to the predictor variables

In [ ]:
df_train.corr(method='pearson').head(1)
# df_train.corr(method='spearman')
# df_train.corr(method='kendall')

In [ ]:

df_train.drop('salary', axis=1).corr()

fig, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(df_train.drop('salary', axis=1).corr(method='kendall'),
            annot=True, ax=ax, linewidths=0.1)
plt.title('Correlation of featues');

### Is there any multicollinearity or other problems that may be a problem in the multiple regression

What the regression model is going to do is to disentangle the individual effects of years worked and years rank on salary. Given the opportunity that the regression model does tease apart these individual effects, such individual effects become obscured. Multicolinearity happens when the X variables are themselves related.

This is a perfect description for multicolinearity, if **one year a experience is added**, the likelihood that one stays in that **rank for one more year** is close to 100%, similarly with **market value**, it is common knowledge that years of experience do actually increase the market value of an individual, so the regression model will have a very unrealistic way of analysing if the effect on salary was due to an increase in market value or due to one more year of experience or one more year of being in that particular rank. Therefore, drop years worked and years absent(the former because it is highly correlated with male but not with salary


### 4. Model creation of salary prediction based on multivariables

In [ ]:
import statsmodels.formula.api as smf
# create a fitted model: salary vs Years worked _ Market + Years Rank
variables = ['yearsworked' ,'market' ,'yearsrank','Engineering','Finance' ,'Marketing' ,'Jnr_employee', 'Manager']
model= smf.ols(formula='salary ~ yearsworked + market + yearsrank+ Engineering + Finance + Marketing +Jnr_employee + Manager', data=df_train).fit()
print(model.params)
print(color.BOLD + "This model has a higher R-squared (0.816) than the previous model's(0.708), This model provides a better fit to the data than a model that only one variable years worked")
model.summary()

### Model Evaluation

In [ ]:
#model1: predict salary using years worked
df_train1 = model.predict(df_train)
df_train1 = pd.DataFrame(df_train1)

#Mapping 2 columns: Observed salary together with Predicted salary
df_train1['Actual Salary'] = df_train['salary']
df_train1= df_train1.rename(columns={0:'Predicted Salary'})
df_train1['Actual Salary'] = df_train1['Actual Salary'].astype(float)
df_train1['Predicted Salary'] = df_train1['Predicted Salary'].astype(float)




### 5.1 Residuals

In [ ]:
#Creating a residual column( a residual vertical distance between a data point and the regression line)
df_train1['Residual'] = (df_train1['Actual Salary'] - df_train1['Predicted Salary'])

#calculating the RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
y_actual1 = df_train1['Actual Salary']
y_predicted1 = df_train1['Predicted Salary']
rmse_train = rmse(df_train.salary, model.predict(df_train[variables]))
rmse_test = rmse(df_test.salary, model.predict(df_test[variables]))
# train_rmse = rmse(df_train.salary, model.predict(df_train[variables]))
# test_rmse = rmse(df_test.salary, model.predict(df_test[variables]))
#standardizing residuals
#Standardized residual = (observed count – expected count) / √expected count
df_train1['Standardised Residuals'] = df_train1.Residual/rmse_train


## Residual plot

In [ ]:
sns.residplot(df_train1['Predicted Salary'],df_train1['Standardised Residuals'], lowess= True)
plt.show()

Data points are pretty symmetrically distributed, tending to cluster towards the middle of the plot and  they’re clustered around y = 0 showing no patterns

### 7. Running the model on the test set

In [ ]:
# Quartiles and IQR:df_test
q75_test, q25_test = np.percentile(df_test['salary'], [75 ,25])
iqr_test = q75_test - q25_test

# Extremes' boundaries: df_test
test_lower_bound = q25_test - iqr_test*1.5
test_upper_bound = q75_test + iqr_test*1.5

#Determining if an entry is an extreme or an outlier:df_test
df_test['Is_it_an_extreme'] = ['Yes' if i < test_lower_bound else 'Yes'
                               if i > test_upper_bound else "No"
                               for i in df_test.salary]
df_test_outliers = df_test[df_test.Is_it_an_extreme == 'Yes']

#Dropping outliers
df_test = df_test[df_test.Is_it_an_extreme != 'Yes']




In [ ]:
#model1: predict salary using years worked
model1= smf.ols(formula='salary ~ yearsworked + market + yearsrank+ Engineering + Finance + Marketing +Jnr_employee + Manager', data=df_test).fit()

df_test1 = model1.predict(df_test)
df_test1 = pd.DataFrame(df_test1)

#Mapping 2 columns: Observed salary together with Predicted salary
df_test1['Actual Salary'] = df_test['salary']
df_test1= df_test1.rename(columns={0:'Predicted Salary'})
df_test1['Actual Salary'] = df_test1['Actual Salary'].astype(float)
df_test1['Predicted Salary'] = df_test1['Predicted Salary'].astype(float)

#Creating a residual column( a residual vertical distance between a data point and the regression line)
df_test1['Residual'] = (df_test1['Actual Salary'] - df_test1['Predicted Salary'])

#standardizing residuals
#Standardized residual = (observed count – expected count) / √expected count
df_test1['Standardised Residuals'] = df_test1.Residual/rmse_test



### 7.1 RMSE Evals

In [ ]:
print('The RMSE of the train data is: ',rmse_train)
print('And The RMSE of the test data is: ',rmse_test,'which shows little evidence of underfitting nor underfitting')

### Conclusion

After such evidence, it is safe to conclude that this is a good model to predict salary.